<h1 align="center" style="margin-bottom: 20px;">Business Cases with Data Science 2024-25</h1>
<h3 align="center" style="margin-top: 20px; margin-bottom: 20px;">Case 4: AI - Powered Chatbot</h3>
<h5 align="center" style="margin-top: 20px; margin-bottom: 0px;">Notebook 1 

### Group B - Members:
- Ana Marta Azinheira | 20240496@novaims.unl.pt
- Bráulio Damba | 20240007@novaims.unl.pt
- Jan-Louis Schneider | 20240506@novaims.unl.pt
- Sofia Jacinto | 20240598@novaims.unl.pt

# Notebook Objective

- This notebook is designed to test our AI-Powered Chatbot. Throughout the notebook, we develop and refine various prompts to interact with our model. To support this notebook, we include a separate .py file containing the core functions used by the chatbot. At the end of the notebook, users will be able to test the chatbot in a simulated environment.

# Importing Libraries and Others

In [7]:
# Imports
import os
import json
import time
import pandas as pd
from openai import AzureOpenAI
from PIL import Image
from IPython.display import Markdown, display
import pickle
from datetime import datetime, timedelta
import fitz
import markdown
from docx import Document  
import sys
import re
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QTextEdit, QPushButton, QLabel, QTextBrowser
from PyQt5.QtCore import Qt, pyqtSignal, QThread

#pip install pymupdf python-docx
#!pip install python-docx
#!pip install streamlit
#pip install PyQt5 openai pillow
#pip install pyqt5-tools
#pip install markdown

In [8]:
# Import methods from utils.py
from utils import (
    create_assistant,
    create_thread,
    check_assistant_exists,
    load_and_upload_files,
    add_message_to_thread,
    display_messages,
    send_message_to_assistant
)

In [9]:
# Set API key and endpoint
api_key = 'yourKEY'
endpoint = 'https://ai-bcds.openai.azure.com/'

In [10]:
# Constants
current_folder = os.getcwd()
data_folder = current_folder
data_folder_full_path = os.path.abspath(data_folder)

assistantFilename = 'AssistantID.TXT'
assistant_id = None
assistant = None
vector_data = 'vector_store.pkl'   # Where uploaded data will be/is saved

displayedMessagesIDs = []

In [11]:
# Initialize the Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint = endpoint,
    api_key= api_key,
    api_version="2024-05-01-preview")

In [12]:
# Load the link_map created, that links every document with a source link
with open("document_links.json", "r", encoding="utf-8") as f:
    doc_link_map = json.load(f)

In [13]:
# Load or upload documents 
if os.path.exists(vector_data):
    with open(vector_data, "rb") as file:
        vector_store = pickle.load(file)
    print("Loaded existing vector store from file.")
else:
    print("Uploading documents and creating vector store...")
    vector_store = load_and_upload_files(client, link_map=doc_link_map)
    print("Documents uploaded.")

Loaded existing vector store from file.


# Rules/Prompts for the Model

In [14]:
# Role of the model
aRole = (
    "És um assistente virtual da seguradora Fidelidade, fiável e rápido, que apoia os colaboradores durante os atendimentos a clientes.\n"
    "O teu objetivo é fornecer respostas claras, corretas e rápidas, ajudando os colaboradores da Fidelidade a responder com confiança.\n"
    "Entendes o contexto da conversa e considera sempre as perguntas anteriores para manter coerência nas respostas.\n"
    "Se o colaborador fizer uma pergunta de seguimento, lembre-se do que foi dito antes.\n"
    "Não interages diretamente com o cliente final, mas atua como um suporte eficiente para os colaboradores.\n"
    "Responde de forma natural, amigável e clara."
)

In [15]:
# open prompt rules txt
with open("prompt_rules.txt", "r", encoding="utf-8") as f:
    prompt_rules = f.read()

In [16]:
# Load or create assistant using existing vector store

# Try load existing assistant
if os.path.exists(assistantFilename):
    with open(assistantFilename, "r") as file:
        assistant_id = file.read().strip()

    # Check if assistant exists in azure
    exists, assistant = check_assistant_exists(client, assistant_id)
    if exists:
        # Load assistant with new role
        assistant = client.beta.assistants.update(
            assistant_id=assistant_id,
            instructions=aRole, 
            tool_resources={    # The documents made available for the model
                "file_search": {
                    "vector_store_ids": [vector_store.id]
                }
            }
        )
        print("Using existing assistant:", assistant_id)

    else:  # If assistant exist but not valid in azure
        print("Assistant ID found, but not valid in API. Creating new one...")
        assistant_id = None   # Marker to create new assistant 

else:   # If no assistant found
    print("ℹNo assistant ID file found. Creating new assistant...")
    assistant_id = None   # Marker to create new assistant

# No valid assistant --> create new one
if assistant_id is None:
    print("Creating new assistant...")
    assistant = create_assistant(client, aRole, assistantFilename)
    assistant_id = assistant.id

    assistant = client.beta.assistants.update(
        assistant_id=assistant.id,
        tool_resources={
            "file_search": {
                "vector_store_ids": [vector_store.id]
            }
        }
    )
    print("New assistant created and linked.")

Using existing assistant: asst_afxDUFnwot8aRxGIbDcmPyiv


In [17]:
# New thread
thread = create_thread(client)

# Testing the Chat Bot

In [18]:
# class making able that pressing enter works as sending
class EnterTextEdit(QTextEdit):
    enter_pressed = pyqtSignal()

    def keyPressEvent(self, event):
        if event.key() == Qt.Key_Return and not (event.modifiers() & Qt.ShiftModifier):
            self.enter_pressed.emit()
        else:
            super().keyPressEvent(event)


#create new background thread to run input/response, so main window runs more fluent
class AssistantWorker(QThread):
    finished = pyqtSignal(str)

    def __init__(self, user_input, prompt_rules, client, thread, assistant, displayedMessagesIDs):
        super().__init__()
        self.user_input = user_input
        self.prompt_rules = prompt_rules
        self.client = client
        self.thread = thread
        self.assistant = assistant
        self.displayedMessagesIDs = displayedMessagesIDs

    def run(self):
        full_prompt = f"{self.prompt_rules}\n\nUser question: {self.user_input}"
        response = send_message_to_assistant(
            self.client, self.thread, self.assistant,
            self.user_input, full_prompt, self.displayedMessagesIDs
        )
        self.finished.emit(response)

# main gui
class AssistantGUI(QWidget):
    # location, size, layout of widgets
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Assistant Chat")
        self.resize(700, 500)
        layout = QVBoxLayout()

        self.output = QTextBrowser()
        self.output.setReadOnly(True)
        self.output.setOpenExternalLinks(True)
        layout.addWidget(QLabel("Chat history"))
        layout.addWidget(self.output)

        self.input = EnterTextEdit()
        self.input.setFixedHeight(50)
        self.input.enter_pressed.connect(self.handle_input)
        layout.addWidget(QLabel("Question"))
        layout.addWidget(self.input)

        self.button = QPushButton("Send")
        self.button.clicked.connect(self.handle_input)
        layout.addWidget(self.button)

        self.setLayout(layout)

    # so links can be clickable
    def convert_links_to_html(self, text):
        # Regex, das eine URL abgrenzt und aufhört bei Satzzeichen oder Leerzeichen
        pattern = r'\b(https?://[^\s<>"\'\]\[)]+)'
    
        def replacer(match):
            url = match.group(1).rstrip('.,;:!?')
            trailing = match.group(1)[len(url):]
            return f'<a href="{url}">{url}</a>{trailing}'
    
        return re.sub(pattern, replacer, text)

    # also for links
    def convert_markdown_to_html(self, text):
        return markdown.markdown(text)

    # make link in response clickable
    def handle_response(self, response):
        response_html = self.convert_markdown_to_html(response)
        self.output.append(f"Assistant: {response_html}")
        print(f"Assistant: {response}")

    # function to handle input
    def handle_input(self):
        user_input = self.input.toPlainText().strip()
        if not user_input:
            return

        self.output.append(f"You: {user_input}")
        self.input.clear()

        if user_input.strip().upper() == "quit":
            self.output.append("Assistant: Obrigado pelo seu contacto. Sempre que precisar estarei aqui. Para que a vida não pare.")
            self.button.setEnabled(False)

        elif user_input.strip().upper() == "NOVA CONVERSA":
            global thread
            self.output.append("Assistant: Iniciando nova conversa...")
            thread = create_thread(client)
            
        else:
            self.output.append("Assistant: (thinking...)")
            QApplication.processEvents()
        
            self.worker = AssistantWorker(user_input, prompt_rules, client, thread, assistant, displayedMessagesIDs)
            self.worker.finished.connect(self.handle_response)
            self.worker.start()

    # to cleanly terminate the session by closing windows
    def closeEvent(self, event):
        self.output.append("Assistant: Session closes, goodbye!")
        print("--session closed--")
    
        # stop the thread also
        if hasattr(self, 'worker') and self.worker.isRunning():
            self.worker.quit()
            self.worker.wait()
    
        event.accept() 


if __name__ == "__main__":
    app = QApplication.instance() or QApplication(sys.argv)
    gui = AssistantGUI()
    gui.show()
    app.exec_()

You: o que sabes sobre fidelidade ppr evoluir?
Thinking...
Assistant: O PPR Evoluir da Fidelidade é um produto inovador que combina proteção e rentabilidade, adaptando-se automaticamente à idade do cliente ao longo do tempo. Aqui estão os principais pontos sobre este plano de poupança-reforma:

- **Componente de Proteção**:
  - Garantia de capital e rendimento, com uma taxa de juro de **2,20%** até junho de 2025.
  - A alocação a esta componente é crescente, atingindo até **60%** conforme a idade do cliente.

- **Componente Ativa**:
  - Focada no aumento da rentabilidade através de uma carteira diversificada, porém sem garantias de capital ou rendimento.

- **Estratégia de Ciclo de Vida**:
  - Início com maior potencial de retorno para idades mais jovens, com diminuição do risco à medida que se aproxima a reforma.

- **Simplicidade**:
  - O cliente não precisa intervir na gestão do produto, pois a alocação é feita automaticamente.

- **Benefícios fiscais**:
  - É também uma oportunidad